### Necessary Python Imports and Setup

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

---
# Cleaning and Organizing the Data for Analysis

Alright, so let's get our data in here and take a look at what we've got.

In [2]:
crime_df = pd.read_csv("vancouver_crime.csv")

crime_df.head()
print(crime_df.shape)

(530652, 12)


530652 crimes!

*For the purposes of this project, I am going to pare down the data a little bit into something more workable. There is some great info in here, but also some extra details that I don't want to deal with while I work with the data.*

### Dropping Extra Columns
First things first, let's get rid of the X, Y, Latitude, and Longitude columns. I am satisfied with letting "Neighborhood" and "Hundred_Block" be representative of the location of the crime.

Next, there is a LOT of "time" information here.
Some of these catagories are great and useful seperately like the Year and Month of the crime.

I am less concerned about which day in the month a crime happened. I believe this is caught well enough by the month column, so I am just going to drop this "Day" column too.

Similarly, I'm less interested in where in the hour crime happened than when in the day a crime happened - I'm going to combine the Minute column into the Hour column. 



In [3]:
df = crime_df

df["HOUR"] = df.apply(
    lambda row:
        round(
            row["HOUR"] + (0.01 * row["MINUTE"] * (100/60)),
            2
        ),
    axis=1
)


df.drop(
    columns = ["X", "Y", "Latitude", "Longitude", "DAY", "MINUTE"],
    inplace = True,
)

df.head()

,TYPE,YEAR,MONTH,HOUR,HUNDRED_BLOCK,NEIGHBOURHOOD
0,Other Theft,2003,5,16.25,9XX TERMINAL AVE,Strathcona
1,Other Theft,2003,5,15.33,9XX TERMINAL AVE,Strathcona
2,Other Theft,2003,4,16.67,9XX TERMINAL AVE,Strathcona
3,Other Theft,2003,4,11.25,9XX TERMINAL AVE,Strathcona
4,Other Theft,2003,4,17.75,9XX TERMINAL AVE,Strathcona


This is looking pretty good, and very workable. 
For the sake of our later visualizations, the minutes have been converted into proportions of the full hours. The small sacrifice in readibility will be worth a smoother visualization, I think.

The last thing we are going to do is make a function to the Month's of the year back into their names, for readability. We will not use this just yet, because for now I'd still like to preserve the order of the months.

In [20]:
monthDict = {
    1:'Jan',
    2:'Feb',
    3:'Mar',
    4:'Apr',
    5:'May',
    6:'Jun',
    7:'Jul',
    8:'Aug',
    9:'Sep',
    10:'Oct',
    11:'Nov',
    12:'Dec'
}

def translate_months(df):
    """
    Takes the dataframe and changes the integer month values to text month values, returning the new dataframe.
    """
    df["MONTH"] = df.apply(
        lambda row:
            monthDict[row["MONTH"]],
        axis = 1
    )
    return df

def translate_month_columns(df):
    """
    Takes the dataframe and changes the integer month columns to text month columns, returning the dataframe.
    """
    df = df.rename(columns = monthDict)
    return df
    

This leaves us with some extremely telling variables to work with!
We have the following types of data in our study:
1. **Type** - a nominal, catagorical variable describing the type of crime committed.
2. **Year** - a nominal, ordinal variable describing the year the crime was committed.
3. **Month** - a nominal, ordinal variable describing the month of the year the crime was committed.
4. **Hour** - a discrete, numerical variable describing the minute of the day that the crime was committed.
5. **Hundred Block** - a nominal, catagorical variable describing the rough block in Vancouver the crime was committed at.
6. **Neighborhood** - a nominal, catagorical variable describing the neighborhood in Vancouver the crime was committed at.
---

*Here we are saving these names into a crime_columns list for later*

In [5]:
crime_columns = list(df.columns)

### Missing Values 
Before working with the data, we need to decide on what to do with the missing values in the data frame.

In [6]:
missing_df = df.isnull().sum()

missing_df

TYPE                 0
YEAR                 0
MONTH                0
HOUR             54362
HUNDRED_BLOCK       13
NEIGHBOURHOOD    56624
dtype: int64

**Since most of the missing data is in the hour and neighborhood columns, we are going to work with two situations. The first the full data frame of our sample space, with all of the crimes ( but we will only be able to use these for our calculations that do not involve the HOUR or NEIGHBORHOOD features.**

**The second will be the smaller subset of data that does include both the hour and neighborhood information. This will not be used for trends and calculations on year/month, but will be used on calculations about neighborhood and hour of day.**

In [7]:
date_df = df[["TYPE", "YEAR", "MONTH", "HUNDRED_BLOCK"]].dropna()
date_df.isnull().sum()
print(date_df.shape)


(530639, 4)


In [8]:
hour_location_df = df.dropna()
hour_location_df.isnull().sum()
print(hour_location_df.shape)

(474015, 6)


For the hour_location_df, we ended up dropping about 11% of our data, which is not great.

*Unfortunately, I don't know how to clean data properly yet, and I'm not sure of the strategies to add back in the hour-of-day, which is not related to any other variables, without totally warping the averages of the HOUR variable, which is barely a numerical data type to begin with.*
*For the sake of this statistics project, we will continue with our slightly smaller dataset when dealing with hour-of-day calculations*

---
# Data Exploration
*Questions for us to try and answer with statistics:*

#### Date (Full Data) questions
1. How does the number of crimes change year to year? Are they getting more or less frequent?
2. Which months are above average in their crime rate? Below average?

#### Neighborhood / Hour Questions
1. Are certain crimes more common at a certain time of day? Is there any correlation between the rate of a certain crime and the time of day?
2. Which neigborhoods have the least reported crimes? Which have the most? 
3. Can we represent the relationship of crime types and neighbourhoods in a data frame?


# Explorations of Yearly Trends and Statistics
*First, let's take a look at some counts of how the months and years relate.*

In [43]:
year_df = date_df.pivot_table(
    columns = "MONTH",
    index = "YEAR",
    values = "HUNDRED_BLOCK",
    aggfunc = "count",
    margins = True,
    fill_value = 0
)

year_df = translate_month_columns(year_df)
year_df = year_df.astype("int")

year_df

MONTH,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,All
YEAR,,,,,,,,,,,,,
2003,4399,3548,3933,4214,4647,4490,4098,4281,3972,4494,3922,3928,49926
2004,3767,3697,4254,4116,4041,3862,3947,4371,4544,4479,4218,3979,49275
2005,3849,3425,3835,3806,3950,3747,3682,3861,3691,3692,3555,3568,44661
2006,3915,3262,3688,3137,3422,3591,3654,3658,3524,3584,3363,3522,42320
2007,3551,2787,3051,3076,3158,3198,3242,3635,3186,3173,2964,2671,37692
2008,2760,2829,3309,2907,3072,2989,2960,3068,2988,3078,2823,2631,35414
2009,2743,2692,2544,2533,2789,2641,2793,2907,2815,2693,2664,2358,32172
2010,2453,2245,2430,2338,2486,2444,2499,2839,2614,2751,2294,2295,29688
2011,2360,2124,2315,2148,2294,2901,2694,2489,2439,2299,2254,2256,28573


In [ ]:
TODO: Some stupid simple formulas for probability.

NEED TO

* learn how to get mean and variance of the data (df.describe()?)